In [2]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [31]:
train_dir = os.path.join('data','with_without')
all_classification = []
all_labels = []

In [39]:
for folder in os.listdir(train_dir):
    for file in os.listdir(os.path.join(train_dir, folder)):
        img_path = os.path.join(train_dir, folder, file)
        img = cv2.imread(img_path,0)
        arr = np.array(img)
        resized_img = cv2.resize(arr,(256,256))
        resized_arr = np.array(resized_img)
        all_labels.append(resized_arr)
        all_classification.append(folder)

In [40]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

encoder = LabelEncoder()

X = np.array(all_labels)
X = X/255
y = np.array(all_classification)
y = encoder.fit_transform(y)

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=42)

In [47]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dropout,BatchNormalization,Dense,Flatten

model =Sequential()
model.add(Conv2D(32,(3,3),input_shape=(X.shape[1],X.shape[2],1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(16,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 dropout_12 (Dropout)        (None, 127, 127, 32)      0         
                                                                 
 conv2d_10 (Conv2D)          (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 62, 62, 32)        0         
 ng2D)                                                           
                                                                 
 dropout_13 (Dropout)        (None, 62, 62, 32)       

In [69]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(X_train,y_train,epochs=2,batch_size=64,validation_data=[X_test,y_test])

Epoch 1/2
95/95 [==============================] - 121s 1s/step - loss: 0.0773 - accuracy: 0.9763 - val_loss: 0.5475 - val_accuracy: 0.8332
Epoch 2/2
95/95 [==============================] - 121s 1s/step - loss: 0.0631 - accuracy: 0.9813 - val_loss: 0.5141 - val_accuracy: 0.8438


In [70]:
from  sklearn.metrics import confusion_matrix
prediction = model.predict(X_test)
prediction = (prediction>0.5).astype(np.int64)
cm = confusion_matrix(y_test,prediction)
print(cm)

48/48 [==============================] - 5s 103ms/step
[[585 147]
 [ 89 690]]


In [71]:
model.save('mask_detection.h5')

c:\Users\ssnow\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import cv2
import numpy as np
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [3]:
from keras.models import load_model

load_model = load_model('mask_detection.h5')

def preprocess_face(face_img):
    gray_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (256, 256))
    resized_img = np.expand_dims(resized_img, axis=-1)
    resized_img = resized_img / 255.0
    resized_img = np.expand_dims(resized_img, axis=0)
    return resized_img


def prediction_function(face_img):
    face_img = preprocess_face(face_img)
    prediction = load_model.predict(face_img)
    return prediction[0][0] > 0.5 

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        if prediction_function(face_roi):
            label = "No mask"
            color = (0, 0, 255) 
        else:
            label = "Mask"
            color = (0, 255, 0)   
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
    cv2.imshow('Result', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


In [4]:
def preprocess_face(face_img):
    gray_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized_img = cv2.resize(gray_img, (256, 256))
    resized_img = np.expand_dims(resized_img, axis=-1)
    resized_img = resized_img / 255.0
    resized_img = np.expand_dims(resized_img, axis=0)
    return resized_img

def predict_mask(face_img):
    
    face_img = preprocess_face(face_img)
    prediction = load_model.predict(face_img)
    return prediction[0][0] > 0.5 

image_path = "D:\Visual studio\Course\Deep Learning\Jupyter Notebook\Face mask detection\data\images\maksssksksss60.png"  # Path to your test image
#image_path = "D:\Visual studio\Course\Deep Learning\Jupyter Notebook\Face mask detection\data\images\maksssksksss48.png"
image = cv2.imread(image_path)

faces = face_cascade.detectMultiScale(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), 1.3, 5)

for (x, y, w, h) in faces:
    face_roi = image[y:y+h, x:x+w]
    if predict_mask(face_roi):
        label = "No Mask"
        color = (0, 0, 255) 
    else:
        label = "Mask"
        color = (0, 255, 0) 

    cv2.putText(image, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)

cv2.imshow('Face Mask Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 [==============================] - 0s 33ms/step
